In [ ]:
using Revise
using ConvexHullPricing
using Gurobi
using Plots
using BenchmarkTools

In [ ]:
ThermalGen = ConvexHullPricing.Utilitaries.ThermalGen(
    MinRunCapacity = [6],
    MaxRunCapacity = [16],
    RampUp = [5],
    RampDown = [5],
    StartUp = [6],
    ShutDown = [6],
    UpTime = [1],
    DownTime = [1],
    NoLoadConsumption = [10],
    MarginalCost = [53],
    FixedCost = [30],
)
instance = ConvexHullPricing.Utilitaries.Instance(
    LostLoad = 3000,
    Load = [6 11 16 11],
    ThermalGen = ThermalGen
)

LP_Relax = ConvexHullPricing.Utilitaries.LP_Relaxation(instance)
T = length(instance.Load)
N_ITER = 100

In [ ]:
LP_Relax

In [ ]:
using JLD2
caBLMdf = load_object("C:\\Users\\Sofiane\\Desktop\\ConvexHullPricing\\results\\tables\\caBLMdf.jld2")

In [ ]:
optimals = load_object("C:\\Users\\Sofiane\\Desktop\\ConvexHullPricing\\results\\tables\\caOptimalDF.jld2")

In [38]:
(caBLMdf.fiterates[1][end - 1] - last(optimals.fiterates[1]))/abs(last(optimals.fiterates[1]))

-6.755697900888386e-8

In [26]:
opt = optimals[1]
fiter = caBLMdf.fiterates[1]

ArgumentError: ArgumentError: syntax df[column] is not supported use df[!, column] instead

In [ ]:
opt, it, fs = ConvexHullPricing.Optimizer.ColumnGeneration(instance, LP_Relax, 1, 1e-5)
@show last(fs)

In [ ]:
((ItBLM, VBLM, oracle_time, solvetime), runtime) = @timed ConvexHullPricing.Optimizer.tBPM(instance, 127548., LP_Relax, .98, 1e-1)

In [ ]:
@show oracle_time
@show solvetime
@show runtime
@show length(ItBLM)

In [ ]:
OptimalBLM, IteratesBLM, ValBLM = ConvexHullPricing.Optimizer.BundleLevelMethod(instance, LP_Relax, N_ITER, .98)

In [ ]:
OptimalDA, IteratesDA, ValDA = ConvexHullPricing.Optimizer.DAdaptation(instance, LP_Relax, N_ITER, 1e-8)

In [ ]:
instance = ConvexHullPricing.Utilitaries.load_data("C:\\Users\\Sofiane\\Desktop\\ConvexHullPricing\\data\\belgian-autumnwd.json")
LP_Relax = ConvexHullPricing.Utilitaries.LP_Relaxation(instance)
T = length(instance.Load)

In [ ]:
OptimalBLM, IteratesBLM, ValBLM = ConvexHullPricing.Optimizer.BundleLevelMethod(instance, LP_Relax, N_ITER, .8)
OptimalDowG, IteratesDowG, ValDowG = ConvexHullPricing.Optimizer.DoWG(instance, LP_Relax, N_ITER, 1.)

In [ ]:
OptimalBPM, IteratesBPM, ValBPM = ConvexHullPricing.Optimizer.BundleProximalMethod(instance, LP_Relax, N_ITER, .98, 1e-1)

In [ ]:
OptimalDA, IteratesDA, ValDA = ConvexHullPricing.Optimizer.DAdaptation(instance, LP_Relax, 100, 1e3)

In [ ]:
OptimalCG, IteratesCG = ConvexHullPricing.Optimizer.ColumnGeneration(instance, N_ITER, 1e-5)
ValCG = []
for price in IteratesCG 
    push!(ValCG, ConvexHullPricing.Utilitaries.fast_oracle(instance, price)[1])
end

In [ ]:
println("BLM : $(last(ValBLM))")
println("BPM : $(last(ValBPM))")
println("DowG : $(last(ValDowG))")
println("DAdapt : $(last(ValDA))")
println("OGM : $(last(ValOGM))")
println("CG : $(last(ValCG))")

In [ ]:
mu = .1
OptimalOGM, IteratesOGM, ValOGM = ConvexHullPricing.Optimizer.OptimizedGradientMethod(instance, LP_Relax, N_ITER, mu)

ValOGM = []
for price in IteratesOGM
    push!(ValOGM, ConvexHullPricing.Utilitaries.fast_oracle(instance, price)[1])
end

In [ ]:
# hline([-850], label = "Optimal Value", linewidth = 1.5)
plot(-ValBLM, label = "Bundle Level Method", linewidth = 1.5)
plot!(-ValBPM, label = "Proximal Level Method", linewidth = 1.5)
plot!(-ValOGM, label = "Optimized Gradient Method", linewidth = 1.5)
plot!(-ValDowG, label = "DoWG", linewidth = 1.5)
plot!(-ValDA, label = "D-Adaptation", linewidth = 1.5)
plot!(-ValCG, label = "Column Generation", linewidth = 1.5)
plot!(size = (1000, 800))